<a href="https://colab.research.google.com/github/Cypher29/AI_repo/blob/main/tokenizing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers torch

In [ ]:
from transformers import AutoModel

In [ ]:
from google.colab import userdata
from huggingface_hub import login
from transformers import AutoTokenizer
from transformers import pipeline
import anthropic

In [ ]:
hf_token = userdata.get('HUGGINGFACE_KEY')
login(hf_token, add_to_git_credential=True)

In [ ]:
config = AutoModel.from_pretrained('microsoft/MAI-DS-R1')

In [ ]:
print(config)

This is sentiment analysis model

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('distilbert/distilbert-base-uncased-finetuned-sst-2-english', trust_remote_code=True)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('google-bert/bert-base-uncased', trust_remote_code=True)

In [ ]:
len(tokenizer)

In [ ]:
print(tokenizer)

In [ ]:
text = "I am excited to show Tokenizers in action to my LLM engineers"
tokens = tokenizer.encode(text)
vocabsize = tokenizer.vocab_size
print("Vocab size : " ,vocabsize)
total_parameters = sum(p.numel() for p in Params.parameters())
print("Total parameters : " ,total_parameters)
tokens

In [ ]:
len(tokens)

In [ ]:
tokenizer.decode(tokens)

In [ ]:
tokenizer.batch_decode(tokens)

In [ ]:
tokenizer.get_added_vocab()

In [ ]:
PHI3_MODEL_NAME = "microsoft/Phi-3-mini-4k-instruct"

In [ ]:
phi3_tokenizer = AutoTokenizer.from_pretrained(PHI3_MODEL_NAME)

text = "I am excited to show Tokenizers in action to my LLM engineers"
print(tokenizer.encode(text))
print()
tokens = phi3_tokenizer.encode(text)
print(phi3_tokenizer.batch_decode(tokens))

In [ ]:
tokenizer.all_special_tokens